In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
df_day=pd.read_csv('day.csv')
df_day.head(5)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [ ]:
df_hour=pd.read_csv('hour.csv')
df_hour.head(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


##### For exploratory data analysis, we will be creating a copy of df_day to perform data cleaning tasks without disturbing the actual dataframe.Using df_day dataframe to discover patterns, trends by day wise.For further hourly pattern analysis,we will use df_hour dataframe later.

## Exploratory Data Analysis

In [ ]:
df_day_ex=df_day.copy()

In [ ]:
df_day_ex.shape

(731, 16)

In [62]:
df_day_ex['yr']=df_day_ex['yr'].map({0:'2011',1:'2012'})
df_day_ex['season'] = df_day_ex['season'].map({ 1: 'Spring', 2 : 'Summer',3:'Fall',4:'Winter'})
df_day_ex['weekday'] = df_day_ex['weekday'].map({0:'Sunday',1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday'})
df_day_ex['mnth']= df_day_ex['mnth'].map({1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',
                   7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})
df_day_ex['weathersit']= df_day_ex['weathersit'].map({1:'Clear',2:'Cloudy',3:'Light Rain and Snow',4:'Heavy Rain'})

Checking whether the mapping method is implemented with out any nan values.

In [63]:
df_day_ex.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [65]:
df_day_ex.rename(columns={"yr": "year", "mnth":"month","weathersit":"weather","temp":"temperature","atemp":"feeling temp","hum":"humidity","cnt":"count"},inplace=True)

In [66]:
df_day_ex

,instant,dteday,season,year,month,holiday,weekday,workingday,weather,temperature,feeling temp,humidity,windspeed,casual,registered,count
0,1,2011-01-01,Spring,2011,January,0,Saturday,0,Cloudy,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,Spring,2011,January,0,Sunday,0,Cloudy,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,Spring,2011,January,0,Monday,1,Clear,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,Spring,2011,January,0,Tuesday,1,Clear,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,Spring,2011,January,0,Wednesday,1,Clear,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,Spring,2012,December,0,Thursday,1,Cloudy,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,Spring,2012,December,0,Friday,1,Cloudy,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,Spring,2012,December,0,Saturday,0,Cloudy,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,Spring,2012,December,0,Sunday,0,Clear,0.255833,0.231700,0.483333,0.350754,364,1432,1796


Dropping off the columns dteday as it represents (year,month,date) where we have seperate columns and instant where it is index from the table.

In [68]:
df_day_ex.drop(['dteday','instant'],axis=1,inplace=True)

In [70]:
df_day_ex.head(5)

,season,year,month,holiday,weekday,workingday,weather,temperature,feeling temp,humidity,windspeed,casual,registered,count
0,Spring,2011,January,0,Saturday,0,Cloudy,0.344167,0.363625,0.805833,0.160446,331,654,985
1,Spring,2011,January,0,Sunday,0,Cloudy,0.363478,0.353739,0.696087,0.248539,131,670,801
2,Spring,2011,January,0,Monday,1,Clear,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,Spring,2011,January,0,Tuesday,1,Clear,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,Spring,2011,January,0,Wednesday,1,Clear,0.226957,0.229270,0.436957,0.186900,82,1518,1600
